In [1]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import csv
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

from __future__ import print_function
import time

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

#!pip install efficient-apriori
from efficient_apriori import apriori

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.dates as mdates

import seaborn as sns

from ydata_profiling import ProfileReport

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

import pyfpgrowth

# Seed para reprodução de resultados
seed = 1
random.seed(seed)
np.random.seed(seed)

In [2]:
#list all csv files only
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

['baker1a_0-299_VS.csv',
 'baker1b_0-299_VS.csv',
 'baker2a_1200-1499_VS.csv',
 'baker2b_1200-1499_VS.csv',
 'baker3a_2700-2999_VS.csv',
 'baker3b_2700-2999_VS.csv',
 'baker4a_5100-5399_VS.csv',
 'baker4b_5100-5399_VS.csv',
 'baker5a_6000-6299_VS.csv',
 'baker5b_6000-6299_VS.csv',
 'baker6a_9000-9299_VS.csv',
 'baker6b_9000-9299_VS.csv',
 'baker7a_9300-9599_VS.csv',
 'baker7b_9300-9599_VS.csv']

In [3]:
df_append = pd.DataFrame()
#append all files together
for file in csv_files:
            df_temp = pd.read_csv(file,na_values=[-99999.25])
            df_append = df_append.append(df_temp, ignore_index=True)
df_append

C:\Users\gustavo.araujo\AppData\Local\Temp\ipykernel_22072\1762955736.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\gustavo.araujo\AppData\Local\Temp\ipykernel_22072\1762955736.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\gustavo.araujo\AppData\Local\Temp\ipykernel_22072\1762955736.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\gustavo.araujo\AppData\Local\Temp\ipykernel_22072\1762955736.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

,TIME (Date and Time),0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),...,9546 (SFS Ext. Function: Pilot Supply),9547 (SFS Ext. Function: Manifold Regulated Supply),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4),9554 (External Pod Command State - Solenoid 5),9555 (External Pod Command State - Solenoid 6),9556 (External Pod Command State - Solenoid 7),9557 (External Pod Command State - Solenoid 8)
0,2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-08T18:00:03.000Z,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-08T18:00:06.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-08T18:00:09.000Z,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-08T18:00:39.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127995,2022-12-10T00:36:03.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127996,2022-12-10T00:36:17.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127997,2022-12-10T00:36:25.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127998,2022-12-10T00:36:45.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Set date as Index
df_append = df_append.set_index('TIME (Date and Time)').drop_duplicates()
df_append = df_append.sort_index(axis=0)
df_append

,0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),1 (Blue SEM-A;B Active/Standby and Online/Offline State),...,9546 (SFS Ext. Function: Pilot Supply),9547 (SFS Ext. Function: Manifold Regulated Supply),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4),9554 (External Pod Command State - Solenoid 5),9555 (External Pod Command State - Solenoid 6),9556 (External Pod Command State - Solenoid 7),9557 (External Pod Command State - Solenoid 8)
TIME (Date and Time),,,,,,,,,,,,,,,,,,,,,
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:01.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:02.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-10T01:57:22.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-10T01:59:17.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-10T02:00:39.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Verificar atributos com muitos missing
percent_missing = df_append.apply(lambda x: x.isnull().sum() * 100 / len(df_append), axis=0).sort_values(ascending=False)
df = pd.DataFrame({'column_name': df_append.columns, 'percent_missing': percent_missing})
df_missing = pd.DataFrame({'column_name': df.index,'percent_missing': percent_missing})
df_no_data = df_missing[percent_missing > 99.999]
df_filtered = df_append.drop(columns= df_no_data['column_name'])
df_filtered

,0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),1305 (SC Manifold Regulator Pressure - Decrease),...,9532 (SmV Ext. Function: Lower Inner Kill Valve - Close),9533 (SmV Ext. Function: Lower Inner Kill Valve - Open),9540 (SFS Ext. Function: Lower Outer Kill Valve - Close),9541 (SFS Ext. Function: Lower Outer Kill Valve - Open),9542 (SFS Ext. Function: Lower Inner Kill Valve - Close),9543 (SFS Ext. Function: Lower Inner Kill Valve - Open),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4)
TIME (Date and Time),,,,,,,,,,,,,,,,,,,,,
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:01.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:02.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-10T01:57:22.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-10T01:59:17.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-10T02:00:39.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Verificar quais atributos mais frequentes da base
df_missing.tail(20)

,column_name,percent_missing
6209 (Diverter - Manifold Regulated Pressure (PT4-1)),6209 (Diverter - Manifold Regulated Pressure (...,96.697810
1409 (Upper Annular Regulator Readback Pressure),1409 (Upper Annular Regulator Readback Pressure),96.597960
9274 (Stack HPHT Pressure - Corrected (PSI)),9274 (Stack HPHT Pressure - Corrected (PSI)),96.597960
1406 (Manifold Regulator Readback Pressure),1406 (Manifold Regulator Readback Pressure),96.533771
9224 (Stack HPHT Pressure - Raw (PSI)),9224 (Stack HPHT Pressure - Raw (PSI)),96.505242
1414 (Lower Anuular Regulator Pilot Pressure),1414 (Lower Anuular Regulator Pilot Pressure),95.948934
9477 (LMRP HPHT Pressure - Corrected (PSI)),9477 (LMRP HPHT Pressure - Corrected (PSI)),95.556665
9277 (LMRP HPHT Pressure - Corrected (PSI)),9277 (LMRP HPHT Pressure - Corrected (PSI)),95.399758
9427 (LMRP HPHT Pressure - Raw (PSI)),9427 (LMRP HPHT Pressure - Raw (PSI)),95.371229
9227 (LMRP HPHT Pressure - Raw (PSI)),9227 (LMRP HPHT Pressure - Raw (PSI)),94.836317


In [7]:
## Substituir por "1" valores que tiveram modificacoes entre registros
df_filtered_full = df_filtered.fillna(0)
df_filtered_changes = df_filtered_full.diff()
df_filtered_changes[df_filtered_changes!=0] = 1
df_filtered_changes = df_filtered_changes.drop('2022-12-08T18:00:00.000Z',axis = 0)
df_filtered_changes

,0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),1305 (SC Manifold Regulator Pressure - Decrease),...,9532 (SmV Ext. Function: Lower Inner Kill Valve - Close),9533 (SmV Ext. Function: Lower Inner Kill Valve - Open),9540 (SFS Ext. Function: Lower Outer Kill Valve - Close),9541 (SFS Ext. Function: Lower Outer Kill Valve - Open),9542 (SFS Ext. Function: Lower Inner Kill Valve - Close),9543 (SFS Ext. Function: Lower Inner Kill Valve - Open),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4)
TIME (Date and Time),,,,,,,,,,,,,,,,,,,,,
2022-12-08T18:00:01.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:02.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:03.000Z,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:03.000Z,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:04.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-10T01:57:22.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-10T01:59:17.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-10T02:00:39.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#Transformando os valores "1" no nome do atributo
for value in df_filtered_changes:
    df_filtered_changes[value] = df_filtered_changes[value].replace([1.0], value)
df_filtered_changes   

,0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),1305 (SC Manifold Regulator Pressure - Decrease),...,9532 (SmV Ext. Function: Lower Inner Kill Valve - Close),9533 (SmV Ext. Function: Lower Inner Kill Valve - Open),9540 (SFS Ext. Function: Lower Outer Kill Valve - Close),9541 (SFS Ext. Function: Lower Outer Kill Valve - Open),9542 (SFS Ext. Function: Lower Inner Kill Valve - Close),9543 (SFS Ext. Function: Lower Inner Kill Valve - Open),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4)
TIME (Date and Time),,,,,,,,,,,,,,,,,,,,,
2022-12-08T18:00:01.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:02.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:03.000Z,0.0,0.0,0.0,0.0,0.0,250 (Test Pod SEM-A Serial Software Process - ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:03.000Z,0.0,0.0,0.0,0.0,0.0,250 (Test Pod SEM-A Serial Software Process - ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:04.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-10T01:57:22.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-10T01:59:17.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-10T02:00:39.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#carregando o dataframe com Nan nos valores "0" e passando para um arquivo csv com uma lista csv de transacoes.
df_filtered_changes.replace(0, np.nan, inplace=True)

df_filtered_changes.to_csv(r'C:\Users\gustavo.araujo\OneDrive - SafeKick\Desktop\pessoal\Pos\ICA\Projeto final\BOP Monitoring\baker_data\associate_names.csv')

# Carregar base como uma lista de itens por transação (não carregaremos NaN -> células em branco)
with open('associate_names.csv') as csvfile:
    transactions = list(csv.reader(csvfile))

del transactions[0]
transactions[0:5] # visualizar primeiras 5 linhas

[['2022-12-08T18:00:01.000Z',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2904 (Manifold Annular Regulator Pilot Pressure)',
  ''

In [10]:
# Eliminando os valores nulos da lista
new_list = []
new_list2 = []
for x in transactions:   
    for y in x:
        if y != '':
            new_list.append(y)
    new_list.pop(0)
    new_list2.append(new_list)
    new_list = []    
new_list2

[['2904 (Manifold Annular Regulator Pilot Pressure)',
  '2922 (60VDC Voltage Readout)'],
 ['1412 (Lower Annular Regulator Readback Pressure)',
  '2922 (60VDC Voltage Readout)'],
 ['250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary)',
  '1412 (Lower Annular Regulator Readback Pressure)'],
 ['250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary)',
  '1418 (Wellhead Connector Regulator Pilot Pressure)'],
 ['1418 (Wellhead Connector Regulator Pilot Pressure)',
  '2907 (Upper Annular Regulator Pilot Pressure)',
  '2909 (Upper Annular Regulator Readback Pressure)'],
 ['1407 (Upper Annular Regulator Pilot Pressure)',
  '1422 (60VDC Voltage Readout)',
  '1439 (Processor Temperature - Remote)',
  '2907 (Upper Annular Regulator Pilot Pressure)',
  '2909 (Upper Annular Regulator Readback Pressure)'],
 ['1407 (Upper Annular Regulator Pilot Pressure)',
  '1422 (60VDC Voltage Readout)',
  '1439 (Processor Temperature - Remote)'],
 ['9010 (Surface Riser Differential Til

In [22]:
#chamando o modelo e passando como parametro a lista de transacoes limpa
itemsets, rules = apriori(new_list2, min_support=0.01, min_confidence=0.8)
print(rules) 

[{9215 (Calc Flexjoint Differential Bearing), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9227 (LMRP HPHT Pressure - Raw (PSI))}, {9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (Stack HPHT Pressure - Corrected (PSI))} -> {9224 (Stack HPHT Pressure - Raw (PSI))}, {9224 (Stack HPHT Pressure - Raw (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9227 (LMRP HPHT Pressure - Raw (PSI))}, {9224 (Stack HPHT Pressure - Raw (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9274 (Stack HPHT Pressure - Corrected (PSI))}, {9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (Stack HPHT Pressure - Corrected (PSI))} -> {9277 (LMRP HPHT Pressure - Corrected (PSI))}, {9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (Stack HPHT Pressure - Corrected (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9224 (Stack HPHT Pressure - Raw (PSI))}, {9224 (Stack HPHT Pressure - Raw (PSI)), 9274 (Stack HPHT Pressure - Corrected (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9227 (LMRP HPHT Pre

In [23]:
# imprimir regras ordenadas por lift!
Lift_rules = []
for rule in sorted(rules, key=lambda rule: rule.lift, reverse=True):
    Lift_rules.append(rule)
    #print(rule) 
Lift_rules

[{9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (Stack HPHT Pressure - Corrected (PSI))} -> {9224 (Stack HPHT Pressure - Raw (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))},
 {9224 (Stack HPHT Pressure - Raw (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (Stack HPHT Pressure - Corrected (PSI))},
 {9224 (Stack HPHT Pressure - Raw (PSI)), 9227 (LMRP HPHT Pressure - Raw (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9274 (Stack HPHT Pressure - Corrected (PSI))},
 {9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (Stack HPHT Pressure - Corrected (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9224 (Stack HPHT Pressure - Raw (PSI))},
 {9224 (Stack HPHT Pressure - Raw (PSI)), 9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9274 (Stack HPHT Pressure - Corrected (PSI))},
 {9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (Stack HPHT Pressure - Corrected (PSI))} -> {9224 (Stack HPHT Pressure - Raw (PSI))},
 {9224 (Stack HPHT Pre

In [24]:
#Convertendo a lista de listas em um dataframe
Df_rules = pd.DataFrame(Lift_rules)
Df_rules
#Resultados foram muito obvios! Os atributos eram os mesmo valores so que corrigidos

,0
0,"{9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (..."
1,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9277 ..."
2,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9227 ..."
3,"{9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (..."
4,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9277 ..."
5,"{9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (..."
6,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9227 ..."
7,"{9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (..."
8,"{9215 (Calc Flexjoint Differential Bearing), 9..."
9,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9274 ..."


In [26]:
#vamos tentar com outros parametros. Vamos diminuir a confiabilidade para termos maior amostra
itemsets, rules = apriori(new_list2, min_support=0.01, min_confidence=0.5)
print(rules) 

[{1417 (Pod Supply Pressure)} -> {1408 (Pod Pressure)}, {1408 (Pod Pressure)} -> {1417 (Pod Supply Pressure)}, {1438 (Processor Temperature - Local)} -> {1422 (60VDC Voltage Readout)}, {1439 (Processor Temperature - Remote)} -> {1422 (60VDC Voltage Readout)}, {2904 (Manifold Annular Regulator Pilot Pressure)} -> {2922 (60VDC Voltage Readout)}, {2907 (Upper Annular Regulator Pilot Pressure)} -> {2922 (60VDC Voltage Readout)}, {2919 (Wellhead Connector Regulator Readback Pressure)} -> {2922 (60VDC Voltage Readout)}, {2938 (Processor Temperature - Local)} -> {2922 (60VDC Voltage Readout)}, {2939 (Processor Temperature - Remote)} -> {2922 (60VDC Voltage Readout)}, {9217 (Subsea Differential Bearing)} -> {9215 (Calc Flexjoint Differential Bearing)}, {9274 (Stack HPHT Pressure - Corrected (PSI))} -> {9224 (Stack HPHT Pressure - Raw (PSI))}, {9224 (Stack HPHT Pressure - Raw (PSI))} -> {9274 (Stack HPHT Pressure - Corrected (PSI))}, {9277 (LMRP HPHT Pressure - Corrected (PSI))} -> {9227 (LMRP 

In [27]:
Lift_rules = []
for rule in sorted(rules, key=lambda rule: rule.lift, reverse=True):
    Lift_rules.append(rule)
#Convertendo a lista de listas em um dataframe
Df_rules = pd.DataFrame(Lift_rules)
Df_rules
#Resultados novos com a nova confiabilidade.

,0
0,"{9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (..."
1,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9277 ..."
2,"{9274 (Stack HPHT Pressure - Corrected (PSI)),..."
3,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9227 ..."
4,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9227 ..."
5,"{9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (..."
6,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9277 ..."
7,"{9227 (LMRP HPHT Pressure - Raw (PSI)), 9274 (..."
8,{9217 (Subsea Differential Bearing)} -> {9215 ...
9,"{9224 (Stack HPHT Pressure - Raw (PSI)), 9227 ..."


In [35]:
#vamos tentar com outros parametros. Vamos diminuir o min_support para mais associacoes
itemsets, rules = apriori(new_list2, min_support=0.002, min_confidence=0.5)
Lift_rules = []
for rule in sorted(rules, key=lambda rule: rule.lift, reverse=True):
    Lift_rules.append(rule)
#Convertendo a lista de listas em um dataframe
Df_rules = pd.DataFrame(Lift_rules)
Df_rules
#Maior quantidade de resultados, mas alguns com nao tanta confiabilidade

,0
0,{1341 (SC 5K & Auxilary Stab - Extend)} -> {13...
1,{1335 (SC Pipe Rams [Cavity 5] - Open)} -> {13...
2,{1343 (SC Blue BOP Accumulator Charge/Dump Val...
3,{1342 (SC Casing Shear Rams [Cavity 3] - Open)...
4,{1345 (SC Blind Shear Rams [Cavity 2] - Open)}...
...,...
624,{1404 (Manifold Annular Regulator Pilot Pressu...
625,{2904 (Manifold Annular Regulator Pilot Pressu...
626,{1419 (Wellhead Connector Regulator Readback P...
627,{2907 (Upper Annular Regulator Pilot Pressure)...


In [36]:
#Imprimindo o resultado em um arquivo csv.
Df_rules.to_csv(r'C:\Users\gustavo.araujo\OneDrive - SafeKick\Desktop\pessoal\Pos\ICA\Projeto final\BOP Monitoring\association_apriori.csv')